# Filtering inventory plots data using clustering procedure

Goal: from dataset obtained from satelite and radar imagery (spectral and terrain characteristics and their derrivatives) we want to obtain as mush of representative data as possible.
To do this on the data with different buffer-zone acquired by multiplying of radius of the invenotry plot, we perform clustering procedure.

1) K-means
* from the whole dataset we select only those records which correposponds to the largest cluster
* clustering procedure is performed on the non-correlated features
* number of clusters is selected autimatically according to the "elbow rule"
* clustering is performed by each plot

1.1. Attempt as it is <br>
1.2. Feature selection step using PCA. From first two components features with explained variation under third quartile was choosen.


In [1]:
#libraries

import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

from shapely import affinity
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, IsolationForest
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import make_scorer

from yellowbrick.cluster import KElbowVisualizer

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
#dataset

test = pd.read_csv('test_data-х3.csv', index_col=0)
test.head(2)

,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B09,...,NDVI,EVI,MSAVI,NDRE,aspect,slope,wetnessindex,sink,key,class
1,0.1126,0.08185,0.06775,0.04215,0.0656,0.16065,0.19045,0.17490,0.20245,0.06135,...,0.611610,0.260078,0.239022,0.454470,0.744224,0.034715,0.863035,0.75347,0,7
2,0.1126,0.08360,0.06815,0.04220,0.0656,0.16065,0.19045,0.17635,0.20245,0.06135,...,0.613818,0.262498,0.241437,0.457739,0.744224,0.034715,0.863035,0.75347,0,7


## K-means

In [4]:
#setting correlation threshold
correlation_threshold = 0.7

In [5]:
# attaching clusters to each row according to the number of plot
def get_cluster_pixels(data:pd.DataFrame, key: int = 1)->pd.DataFrame: 
    try:
        attmpt = data[data.key == key]
        attmpt_c = attmpt.drop(columns = ['key', 'class']).corr().abs() #'index',
        #attmpt.corr().style.background_gradient(cmap="Blues")

        # Select upper triangle of correlation matrix
        upper = attmpt_c.where(np.triu(np.ones(attmpt_c.shape), k=1).astype(bool))

        # Find features with correlation greater than 0.95
        to_drop = [column for column in upper.columns if any(upper[column] > correlation_threshold)]

        # Drop features 
        attmpt_ = attmpt.drop(to_drop, axis=1)#, inplace=True)

        #preprocessing of the data
        #from sklearn.preprocessing import StandardScaler
        scaler = StandardScaler()
        scaler.fit(attmpt_.drop(columns = ['key', 'class']))#'index',
        scaled_data = scaler.transform(attmpt_.drop(columns = ['key', 'class']))#'index',

        #from yellowbrick.cluster import KElbowVisualizer
        model = KMeans()
        # k is range of number of clusters.
        visualizer = KElbowVisualizer(model, k=(1,8), timings= True)
        visualizer.fit(scaled_data)        # Fit data to visualizer
        plt.close()

        kmeans_model = KMeans(n_clusters = visualizer.elbow_value_, random_state=100) # elbow_value_ == number of clusters
        kmeans_model.fit(scaled_data)

        attmpt["clusters"] = kmeans_model.labels_
        attmpt.clusters.value_counts().reset_index()#.duplicated(subset=['clusters'])#.iloc[0,0]
        return attmpt
    except:
        pass

In [6]:
##selection of rows related to most abundant clusters

def get_selection(attmpt:pd.DataFrame)->pd.DataFrame:
    
    cluster_stat = attmpt.clusters.value_counts().to_dict()
    cluster_count = list(cluster_stat.values())
    cluster_non_equal = cluster_count[0]>cluster_count[1]
    if cluster_non_equal:
        target_cluster = list(cluster_stat.keys())[0]
        mask = attmpt.clusters == target_cluster
        data_grol = attmpt.loc[mask]
    else: 
        print('equal cluster')
        data_grol = pd.DataFrame()
    return data_grol

In [7]:
clustered_df = pd.DataFrame()
attmpt = pd.DataFrame()

for item in test.key.unique():
    try:
        attmpt = get_cluster_pixels(test, key=item)
        attmpt = get_selection(attmpt)
        clustered_df = pd.concat([clustered_df, attmpt])
        print(item) # mute at the final stage, needed for the check
    except:
        pass

0
1
2
3
4
5
6
7
8
10
12
13
14
15
16
17
18
19
equal cluster
20
21
22
23
24
25
26
27
28
30
31
32
33
34
35
36
equal cluster
37
38
39
40
41
42
equal cluster
43
44
45
46
47
equal cluster
48
49
50
51
52
53
54
55
56
57
58
59
equal cluster
60
61
equal cluster
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
122
123
124
125
126
127
equal cluster
128
129
130
132
133
134
135
136
137
138
139
141
142
143
144
equal cluster
145
146
147
148
149
150
151
152
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
equal cluster
182
183
184
185
186
187
188
equal cluster
189
190
191
192
193
194
195
196
197
198
199
200
201
equal cluster
202
equal cluster
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
equal cluster
224
225
226
227
228
229
230
231
232
233
equal cluster
234
235
2

In [17]:
print('Rows kept from initial dataset is equal to', round(len(clustered_df)/len(test)*100), '%')

Rows kept from initial dataset is equal to 43 %


## PCA

THIS SCRIPT NEEDS TO BECOME PRETTIER

In [8]:
from sklearn.decomposition import PCA

In [9]:
def get_cluster_pixels_PCA(data:pd.DataFrame, key: int = 1)->pd.DataFrame: 
    try:
        #subsetting
        attmpt_pca_step = data[data.key == key]
        #abjusting columns
        cols_to_drop = ['key', 'class'] #'index',
        attempt_pca = attmpt_pca_step.drop(columns = cols_to_drop)
        
        #getting values and scalling them
        X = attempt_pca.values
        scaler_pca = StandardScaler()
        scaler_pca.fit(X)
        X_scaled = scaler_pca.transform(X)
        
        #performing of principal component analysis
        pca = PCA(n_components=20, random_state=2020)
        pca.fit(X_scaled)
        X_pca = pca.transform(X_scaled)
        
        #selecting columns for clustering
        
        #first component
        stat_eigen = pd.DataFrame(pca.components_, 
                                  columns=attempt_pca.columns).T.loc[:, 0].abs().quantile(0.75)#.mean()#
        pca_results = pd.DataFrame(pca.components_,columns=attempt_pca.columns).T#.loc[mean_eqigen]
        mask_pca = pca_results.loc[:, 0].abs() > stat_eigen #selecting components with loads above third quartile 
        columns_for_clustering = pca_results.loc[mask_pca].index.tolist()
        
        #second component
        stat_eigen = pd.DataFrame(pca.components_,
                                  columns=attempt_pca.columns).T.loc[:, 1].abs().quantile(0.75)#.mean()#
        pca_results = pd.DataFrame(pca.components_,columns=attempt_pca.columns).T#.loc[mean_eqigen]
        mask_pca = pca_results.loc[:, 1].abs() > stat_eigen #selecting components with loads above third quartile
        second_component = pca_results.loc[mask_pca].index.tolist()
        
        #final list of columns for clustering
        columns_for_clustering = columns_for_clustering+second_component
        
        # filtering subset by columns from previous step
        data_for_clustering = attempt_pca.loc[:, columns_for_clustering]
        
        #BACK TO K-MEANS
         
        #preprocessing of the data
        #from sklearn.preprocessing import StandardScaler
        scaler = StandardScaler()
        scaler.fit(data_for_clustering)#'index',
        scaled_data = scaler.transform(data_for_clustering)#'index',

        #from yellowbrick.cluster import KElbowVisualizer
        model = KMeans()
        # k is range of number of clusters.
        visualizer = KElbowVisualizer(model, k=(1,8), timings= True)
        visualizer.fit(scaled_data)        # Fit data to visualizer
        plt.close()

        kmeans_model = KMeans(n_clusters = visualizer.elbow_value_, random_state=100) # elbow_value_ == number of clusters
        kmeans_model.fit(scaled_data)

        attmpt_pca_step["clusters"] = kmeans_model.labels_
        #attmpt.clusters.value_counts().reset_index()#.duplicated(subset=['clusters'])#.iloc[0,0]
        return attmpt_pca_step
    except:
        pass

In [10]:
##selection of rows related to most abundant clusters

def get_selection_PCA(attmpt_pca_step:pd.DataFrame)->pd.DataFrame:
    
    cluster_stat = attmpt_pca_step.clusters.value_counts().to_dict()
    cluster_count = list(cluster_stat.values())
    cluster_non_equal = cluster_count[0]>cluster_count[1]
    if cluster_non_equal:
        target_cluster = list(cluster_stat.keys())[0]
        mask = attmpt_pca_step.clusters == target_cluster
        data_grol = attmpt_pca_step.loc[mask]
    else: 
        print('equal cluster')
        data_grol = pd.DataFrame()
    return data_grol

In [11]:
clustered_df_PCA = pd.DataFrame()
attmpt = pd.DataFrame()

for item in test.key.unique():
    try:
        attmpt = get_cluster_pixels_PCA(test, key=item)
        attmpt = get_selection_PCA(attmpt)
        clustered_df_PCA = pd.concat([clustered_df_PCA, attmpt])
        print(item) # mute at the final stage, needed for the check
    except:
        pass

0
1
2
3
4
5
6
7
8
9
10
12
13
14
15
16
17
18
19
20
21
22
equal cluster
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
equal cluster
41
42
43
44
45
46
47
48
49
equal cluster
50
equal cluster
51
52
53
54
55
56
57
58
59
60
61
62
equal cluster
63
64
65
66
67
68
69
70
71
72
equal cluster
73
74
75
76
77
78
79
80
81
82
83
equal cluster
84
equal cluster
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
equal cluster
108
109
110
111
112
113
equal cluster
114
equal cluster
115
116
117
118
119
120
122
123
124
125
126
equal cluster
127
128
129
equal cluster
130
132
133
equal cluster
134
135
136
137
equal cluster
138
139
141
142
143
144
145
146
147
148
149
150
151
152
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
equal cluster
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
equal cluster
193
194
195
196
197
198
199
200
equal cluster
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
2

In [16]:
print('Rows kept from initial dataset is equal to', round(len(clustered_df_PCA)/len(test)*100), '%')

Rows kept from initial dataset is equal to 45 %


🤸